## 实现cache友好，进行数据随机打乱，然后重新写入硬盘

In [2]:
import os
import math
import random
import numpy as np
import pandas as pd

In [9]:
dfs = []
for i in range(1, 6):
    df = pd.read_csv('./data/train' + str(i) + '.csv', header=None)
    lb = pd.read_csv('./data/label' + str(i) + '.csv', header=None)
    df[13] = lb[0]
    dfs.append(df)
    
df = pd.concat(dfs)

In [10]:
print(df)

               0      1      2      3      4      5      6         7   \
0       -0.221230 -218.0  -15.0  151.0   -1.0  -74.0  -65.0  0.458700   
1       -0.505680    2.0 -218.0  -15.0  151.0   -1.0  -74.0 -0.490380   
2       -0.073529  -10.5    2.0 -218.0  -15.0  151.0   -1.0  0.435190   
3        1.213900  -54.0  -10.5    2.0 -218.0  -15.0  151.0  0.849830   
4        0.482010   45.0  -54.0  -10.5    2.0 -218.0  -15.0  0.363640   
5       -0.247710   -5.0   45.0  -54.0  -10.5    2.0 -218.0  0.958160   
6       -0.041667 -343.0   -5.0   45.0  -54.0  -10.5    2.0 -0.266360   
7       -0.404760 -326.0 -343.0   -5.0   45.0  -54.0  -10.5 -0.905350   
8       -0.696840  -47.0 -326.0 -343.0   -5.0   45.0  -54.0  0.763820   
9        0.109760   -1.0  -47.0 -326.0 -343.0   -5.0   45.0  0.825610   
10       0.250000  -46.0   -1.0  -47.0 -326.0 -343.0   -5.0  0.644110   
11      -0.361840 -328.0  -46.0   -1.0  -47.0 -326.0 -343.0 -0.336490   
12       1.496500 -246.0 -328.0  -46.0   -1.0  -47.

In [11]:
df = df.sample(frac=1, random_state=random.randint(0,100))
df.to_csv('./data/train.csv')

In [12]:
print(df)

               0      1      2      3      4      5      6         7   \
1318798  0.166670  152.0   -1.0  -10.0   59.0   58.0   11.0 -0.494180   
1489300  0.481130   48.0   -7.0  -20.0   -4.0   73.0  -18.0 -0.583050   
1085860 -0.237700  -16.0  131.0  149.0  442.0   -6.0  137.0 -0.865350   
1666954 -0.587630  -93.0    9.0  648.0  108.0 -158.0  177.0  0.734660   
617569   0.119050  -35.0   10.0  -28.0  204.0   80.0   75.0 -0.643560   
1490234 -0.088235  -43.0  -36.0 -213.0  -18.0    1.0   33.0 -0.269010   
1026533 -0.388890    5.0    2.0    3.0    6.0    0.0   -7.0  0.233850   
240479   0.448720    1.0    4.0   97.0  -33.0    1.0  -31.0  0.533330   
432147  -0.345070 -157.0  -25.0 -334.0  -66.0  -97.0  -26.0 -0.622000   
528424   0.071429   12.0   25.0  -57.0  -63.0    1.0   -3.0  0.421830   
1987514 -0.300000  -17.0    2.0   59.0   52.0  -27.0  -39.0 -0.469770   
1524516  0.010000 -261.0   -5.0  -34.0  -25.0   -6.0  158.0  0.024299   
1485488 -0.456520   11.0  -13.0    5.0  -19.0 -183.

后续只需要从硬盘读入即可

In [4]:
df = pd.read_csv('./data/train.csv')
df = df.drop(['Unnamed: 0'],axis=1)
print(df[0:5])

         0      1      2      3      4      5      6        7        8  \
0  0.16667  152.0   -1.0  -10.0   59.0   58.0   11.0 -0.49418  0.93496   
1  0.48113   48.0   -7.0  -20.0   -4.0   73.0  -18.0 -0.58305 -0.63226   
2 -0.23770  -16.0  131.0  149.0  442.0   -6.0  137.0 -0.86535 -0.81214   
3 -0.58763  -93.0    9.0  648.0  108.0 -158.0  177.0  0.73466 -0.96660   
4  0.11905  -35.0   10.0  -28.0  204.0   80.0   75.0 -0.64356 -0.52764   

         9       10       11        12   13  
0  0.85294 -0.84805 -0.84892 -0.960140 -0.2  
1 -0.62806 -0.61551 -0.63894 -0.861110  0.0  
2 -0.85055  0.64444 -0.21071 -0.197740  0.0  
3  0.81513  0.72252  0.48112  0.098266 -0.4  
4 -0.54987 -0.49630  0.60239  0.477520 -0.4  


## 单/多个进程生成单颗决策树
对于每一棵决策树的生成，使用mse作为评判标准，当整棵树的mse指数不再下降，停止分支；

max_depth参数控制树的深度；

min_samples_split参数控制每个节点分裂最少需要的样本数，min_samples_leaf参数控制每个节点最小样本树，这两个参数可以防止发生过拟合；

min_samples_split，min_samples_leaf越大越不容易过拟合，但min_samples_split>min_samples_leaf；

parallel参数控制并行化，为True时并行，使用多进程寻找最佳分割属性/特征，每个进程针对一个特征寻找最佳分割点，再由主进程选出最佳分割特征

In [5]:
import pandas as pd
import os
import numpy as np
from multiprocessing import Pool, cpu_count, freeze_support
import datetime

In [4]:
class DecisionTree(object):
    def __init__(self, feature_rate=1.0, max_depth=None, min_leaf_size=1, parallel=False):
        self._x = None
        self._y = None
        self._col_names = None
        self._feature_rate = feature_rate
        self._max_depth = max_depth
        self._min_leaf_size = min_leaf_size
        self._parallel = parallel

        self._split_feature = None
        self._split_point = None
        self._mse_score = None
        self._val = None

        self._left_node = None
        self._right_node = None

    def fit(self, x, y):
        # 训练决策树，参数说明如下
        # x，类型为dataFrame，列数为特征数
        # y，类型为dataFrame或者list，列数为1
        self._x = x
        self._y = y
        if type(y) != np.ndarray:
            self._y = np.array(self._y)
        self._col_names = self._x.columns
        self._val = np.mean(self._y)
        self._mse_score = np.power(self._y-self._val, 2).sum()/len(self._y)

        # 如果到达一定深度，停止分裂
        if (self._max_depth is not None) and self._max_depth < 2:
            return self

        # 随机选取feature_rate*col_nums个属性（避免过拟合），选出其中最优的
        feature_num = int(self._feature_rate*len(self._col_names))
        features = np.random.choice(len(self._col_names), feature_num)
        if self._parallel:
            self.find_split_parallel(features)
        else:
            for feature in features:
                tmp_split_feature, tmp_split_point, tmp_mse_score = self.find_best_split_point(feature)
                if tmp_mse_score < self._mse_score:
                    self._split_feature = tmp_split_feature
                    self._split_point = tmp_split_point
                    self._mse_score = tmp_mse_score

        # 如果是寻找最优属性过程中，没能够完成分裂（原因很多种，可能是min_leaf_size限制、mse无法减小限制等），说明该节点是叶节点
        if (self._split_feature is None) or (self._split_point is None):
            return self

        # 选出最优的分割属性、分割点后分裂节点，递归生成子节点
        left_nodes = np.nonzero(np.array(self._x.iloc[:, self._split_feature]) < self._split_point)[0]
        right_nodes = np.nonzero(np.array(self._x.iloc[:, self._split_feature]) >= self._split_point)[0]
        max_depth = self._max_depth - 1 if self._max_depth is not None else None
        self._left_node = DecisionTree(feature_rate=self._feature_rate, max_depth=max_depth,
                                       min_leaf_size=self._min_leaf_size)
        self._left_node.fit(self._x.iloc[left_nodes], self._y[left_nodes])
        self._right_node = DecisionTree(feature_rate=self._feature_rate, max_depth=max_depth,
                                        min_leaf_size=self._min_leaf_size)
        self._right_node.fit(self._x.iloc[right_nodes], self._y[right_nodes])

        return self

    def find_split_parallel(self, features):
        # 并行化寻找最佳分割点
        workers = cpu_count()
        try:
            workers = min(workers, len(features))
        except Exception:
            print('please input correct n_job')
        pool = Pool(processes=workers)
        result = []
        for feature in features:
            result.append(pool.apply_async(self.find_best_split_point, (feature, )))
        # 关闭进程池，等待子进程退出后，拿出结果
        pool.close()
        pool.join()
        # 由于开启不同进程训练树时会发生tree变量的拷贝（而不是引用），所以最后还需要将结果值赋给trees
        for res in result:
            item = res.get()
            if item[2] < self._mse_score:
                self._split_feature = item[0]
                self._split_point = item[1]
                self._mse_score = item[2]

    def find_best_split_point(self, feature):
        # 根据选定特征feature寻找最佳分割点

        points = list(np.argsort(self._x.iloc[:, feature]))
        start = self._x.iloc[points[self._min_leaf_size-1], feature]

        y_square_sum = np.power(self._y, 2).sum()
        y_sum = self._y.sum()
        y_n = len(self._y)
        left_square_sum = np.power(self._y[0:self._min_leaf_size], 2).sum()
        left_sum = self._y[0:self._min_leaf_size].sum()
        left_n = self._min_leaf_size
        right_square_sum = y_square_sum - left_square_sum
        right_sum = y_sum - left_sum
        right_n = y_n - left_n

        tmp_split_feature = None
        tmp_split_point = None
        tmp_mse_score = self._mse_score
        for i in range(self._min_leaf_size, len(self._y)-self._min_leaf_size+1):
            xi, yi = self._x.iloc[points[i], feature], self._y[points[i]]
            tmp = 0
            if xi == start:
                tmp = 1
            if tmp == 0:
                left_score = self.calculate_mse(left_square_sum, left_sum, left_n)
                right_score = self.calculate_mse(right_square_sum, right_sum, right_n)
                score_after_split = left_score*(left_n/y_n) + right_score*(right_n/y_n)
                # 必须要比当前的mse score小才能分裂，不然就有可能出现y都一样但是仍然分裂的状况，因为此时score是相等的
                if score_after_split < tmp_mse_score:
                    tmp_split_feature = feature
                    tmp_split_point = xi
                    tmp_mse_score = score_after_split
            left_square_sum = left_square_sum + yi ** 2
            left_sum = left_sum + yi
            left_n = left_n + 1
            right_square_sum = right_square_sum - yi ** 2
            right_sum = right_sum - yi
            right_n = right_n - 1
            start = xi
        return tmp_split_feature, tmp_split_point, tmp_mse_score

    def predict(self, x):
        # 预测决策树的参数如下：
        # x，类型为array或者dataFrame，其中的每一项表示一个预测对象的特征
        if type(x) != np.ndarray:
            x = np.array(x)
        return [self.predict_one(one) for one in x]

    def predict_one(self, one):
        if (self._split_feature is None) or (self._split_point is None):
            return self._val
        if one[self._split_feature] < self._split_point:
            return self._left_node.predict_one(one)
        return self._right_node.predict_one(one)

    def __repr__(self):
        attr = 'sample size: {} \t value: {} \t mse score: {}\n'.format(len(self._y), self._val, self._mse_score)
        if (self._split_feature is not None) and (self._split_point is not None):
            attr = attr + 'split feature: {} \t split point: {}\n'.format(self._split_feature, self._split_point)
        return attr

    @staticmethod
    def calculate_mse(y_square_sum, y_sum, y_n):
        return (y_square_sum/y_n) - (y_sum**2)/(y_n**2)

单棵树测试

In [8]:
dt = DecisionTree(feature_rate=0.4, min_leaf_size=5)

In [9]:
dt.fit(df.iloc[0:100000, 0:13], np.array(df.iloc[0:100000, 13]))

sample size: 100000 	 value: -0.0027179999999999995 	 mse score: 0.3426780942286538
split feature: 7 	 split point: 0.020381

In [14]:
dt.predict([[0,2,3,4,5,1,2,3,4,5,1,2,3], [21,22,3,-114,-5,1,2,1,4,5,1,2,3]])

[1.0250000000000001, 0.62]

## 使用多进程实现并行生成多棵决策树
使用参数n_job调整并行程度，表示使用的并行进程个数，n_job是多少，就是用多少个并行进程生成决策树

使用参数n_tree调整森林规模，n_tree表示最终森林中决策树的数量，n_tree/n_job表示每个进程应该生成的决策树数量

注：由于Windows下multiprocessing库直接在jupyter下调用会出错，所以实际上自己是将下面这段代码放到同目录的RF.py中，然后导入调用；
但是为了展示，这里也给出代码

In [10]:
class RandomForest(object):
    def __init__(self, feature_rate=1.0, max_depth=None, min_leaf_size=1, n_job=None, n_tree=1, sample_size=None):
        self._feature_rate = feature_rate
        self._max_depth = max_depth
        self._min_leaf_size = min_leaf_size
        self._n_job = n_job
        self._n_tree = n_tree
        self._sample_size = sample_size

        self._trees = [DecisionTree(self._feature_rate, self._max_depth, self._min_leaf_size) for i in
                       range(self._n_tree)]
        self._x = None
        self._y = None

    def fit(self, x, y):
        # 训练森林，参数说明如下
        # x，类型为dataFrame，列数为特征数
        # y，类型为dataFrame，列数为1
        self._x = x
        self._y = y
        if self._n_job:
            self.fit_tree_parallel()
        else:
            for tree in self._trees:
                self.fit_one_tree(tree, self._trees.index(tree))

    def fit_tree_parallel(self):
        # 并行训练森林中的树
        workers = cpu_count()
        try:
            workers = min(workers, self._n_job)
        except Exception:
            print('please input correct n_job')
        pool = Pool(processes=workers)
        result = []
        for tree in self._trees:
            result.append(pool.apply_async(self.fit_one_tree, (tree, self._trees.index(tree))))
        # 关闭进程池，等待子进程退出后，进行赋值
        pool.close()
        pool.join()
        # 由于开启不同进程训练树时会发生tree变量的拷贝（而不是引用），所以最后还需要将结果值赋给trees
        self._trees = [res.get() for res in result]

    def fit_one_tree(self, tree, index):
        # 训练单颗树的参数如下：
        # tree，单棵树的引用，但被其他进程调用时是拷贝
        # index，树的序号
        start = datetime.datetime.now()
        print('fit the {} th tree'.format(index))
        # 首先进行有放回的随机筛选，选出sample_size个样本
        sample_indexes = np.sort(np.random.choice(len(self._x), self._sample_size))
        # 然后调用DecisionTree.fit进行训练
        tree.fit(self._x.iloc[sample_indexes], self._y.iloc[sample_indexes])
        print('{} th tree fit over, time used: {}'.format(index, (datetime.datetime.now()-start).seconds))
        return tree

    def predict(self, x):
        # 预测决策树的参数如下：
        # x，类型为dataFrame，其中的每一项表示一个预测对象的特征
        all_predictions = [tree.predict(x) for tree in self._trees]
        return np.mean(all_predictions, axis=0)

随机森林测试

In [11]:
# This two line are jupyter magical method which use to auto reload the package you changed
%load_ext autoreload
%autoreload 2

In [13]:
import RF
from multiprocessing import Pool, cpu_count, freeze_support
# 这里其实是有输出提示（当前正在生成第几棵树，用时多少，但是由于代码在RF.py中，因此只能在jupyter的命令行那里看到）
if __name__ == "__main__":
    freeze_support()
    rf = RF.RandomForest(feature_rate=0.3, max_depth=32, min_leaf_size=2, n_job=4, n_tree=10, sample_size=50000)
    rf.fit(df.iloc[0:100000, 0:13], df.iloc[0:100000, 13])

In [14]:
rf.predict([[1,2,3,4,5,1,2,3,4,5,1,2,3], [1,2,3,14,5,1,2,1,4,5,1,2,3]])

array([0.29666667, 0.29666667])

## 使用r2 score计算得分

In [16]:
def r2_score(preds, labels):
    assert len(preds) == len(labels)
    y_mean = np.array(labels).mean()
    SSR = (np.power(np.array(preds) - np.array(labels), 2)).sum()
    SST = (np.power(np.array(labels) - y_mean, 2)).sum()
    print('score', 1-SSR/SST)

算分测试

In [17]:
r2_score(rf.predict(df.iloc[0:100000, 0:13]), np.array(df.iloc[0:100000, 13]))

score 0.2095079674884449


即在训练集上得分为0.21，接下来在测试集进行预测，并将结果写入文件，然后提交到竞赛链接

## 使用训练数据训练随机森林，然后做出预测并将结果写入磁盘

In [18]:
import RF
from multiprocessing import Pool, cpu_count, freeze_support
# 这里其实是有输出提示（当前正在生成第几棵树，用时多少，但是由于代码在RF.py中，因此只能在jupyter的命令行那里看到）
if __name__ == "__main__":
    freeze_support()
    rf = RF.RandomForest(feature_rate=0.3, max_depth=32, min_leaf_size=50, n_job=8, n_tree=200, sample_size=2000000)
    rf.fit(df.iloc[:, 0:13], df.iloc[:, 13])

    preds = []
    for i in range(1, 7):
        test_data = pd.read_csv('./data/test'+str(i)+'.csv', header=None)
        tmp = rf.predict(test_data)
        preds.extend(tmp)

    ids = np.arange(1, len(preds)+1)
    res = pd.DataFrame({'id':ids, 'Predicted':preds})
    res.to_csv('./sub.csv', index=False)

In [19]:
submission = pd.read_csv('./sub.csv')
print(submission)

                id  Predicted
0                1  -0.181492
1                2  -0.123607
2                3  -0.156439
3                4  -0.210649
4                5  -0.116018
5                6  -0.042638
6                7  -0.043169
7                8  -0.157669
8                9  -0.157335
9               10  -0.156329
10              11  -0.065739
11              12  -0.159431
12              13  -0.172382
13              14  -0.161974
14              15  -0.195837
15              16  -0.269340
16              17  -0.204745
17              18  -0.165826
18              19  -0.225185
19              20  -0.226774
20              21  -0.130646
21              22  -0.252547
22              23  -0.194690
23              24  -0.161426
24              25  -0.108898
25              26  -0.092830
26              27   0.182336
27              28   0.277736
28              29   0.249279
29              30  -0.286592
...            ...        ...
10915091  10915092   0.145980
10915092  